In [ ]:
!pip install transformers
from transformers import TFBertModel, BertTokenizer

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 16.1MB/s 
     |████████████████████████████████| 890kB 29.1MB/s 
     |████████████████████████████████| 1.1MB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2ae4004041ad61753bb21c7f0fa926f55981fe07d252dd32187ab0819362d7bf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import csv
import string
import re
text = []
labels = []
inputfile = csv.reader(open('./train_2kmZucJ.csv','r'))
for row in inputfile:
  text.append(row[2])
  labels.append(row[1])

In [ ]:
text = text[1:]
temp = text
text = []
for i in temp:
  text.append(i.lower())
labels = labels[1:]
labelss = []
for i in labels:
  i = int(i)
  labelss.append(i)
labels = labelss

In [ ]:
def preprocess(input_text):
  
  ascii = set(string.printable)
  sentences = []
  for sentence in input_text:
    words = []
    sentence = sentence.split(' ')
    x = ['@','%','&']
    for word in sentence:
      c = 0
      for i in x:
        c += word.count(i)
      if c>0:
        continue
      word = word.replace('#','')
      if word[:4] == 'http':
        word = 'http_link'   
      exclude = ['-',':','.','?',')','(','^'] 
      word = ''.join(ch for ch in word if ch not in exclude)
      if len(word)> 15:
        continue
      for l in word.split():
        if l not in ascii:
          continue
      # word = ''.join(ch for ch in word if ch in ascii)
      words.append(word)
    words = ' '.join(words)
    words = re.sub("\d+", "", words)
    sentences.append(words.lower())
  return(sentences)
text_new = preprocess(text)

In [ ]:
tmp = 0
lens = []
for i in text_new:
  tmp += len(i.split())
  lens.append(len(i.split()))
print(tmp/len(text_new))
print(lens[:20])
print(text_new[:20])

15.915277777777778
[13, 15, 15, 17, 23, 11, 10, 27, 13, 14, 25, 19, 16, 19, 16, 12, 15, 11, 14, 22]
['fingerprint pregnancy test http_link android apps beautiful cute health igers iphoneonly iphonesia iphone', 'finally a transparant silicon case  thanks to my uncle  yay sony xperia s sonyexperias… http_link', 'we love this! would you go talk makememories unplug relax iphone smartphone wifi connect http_link', "i'm wired i know i'm george i was made that way ; iphone cute daventry home http_link", "what amazing service! apple won't even talk to me about a question i have unless i pay them $ for their stupid support!", 'iphone software update fucked up my phone big time stupid iphones', 'happy for us  instapic instadaily us sony xperia xperiaz http_link', 'new type c charger cable uk http_link … bay amazon etsy new year rob cross toby young evemun mcmafia taylor spectre  newyear starting  recipes technology samsunggalaxys iphonex', 'bout to go shopping again listening to music iphone jus

In [ ]:
from sklearn.model_selection import train_test_split
train_sents,test_sents, train_labels, test_labels  = train_test_split(text_new,labels,test_size=0.00001)


In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME,do_lower_case = True)
def encoder(sentences):
  ids = []
  for item in sentences:
    encoding = tokenizer.encode_plus(
    item,
    max_length=26,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding['input_ids'])
  return ids
train_ids = encoder(train_sents)
test_ids = encoder(test_sents)  

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
import tensorflow as tf
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
def build_model():
    bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')
    input_word_ids = tf.keras.Input(shape=(26,), dtype=tf.int32, name="input_word_ids")
    #input_mask = tf.keras.Input(shape=(26,), dtype=tf.int32, name="input_mask")    
    embedding = bert_encoder([input_word_ids])
    print(type(embedding[0]))
    dense = tf.keras.layers.Lambda(lambda seq: seq[:, 0, :])(embedding[0])
    dense = tf.keras.layers.Dense(128, activation='relu')(dense)
    dense = tf.keras.layers.Dropout(0.1)(dense)   
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)    
    model = tf.keras.Model(inputs=[input_word_ids], outputs=output)  
    return model
model = build_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


<class 'tensorflow.python.framework.ops.Tensor'>


In [ ]:
model.compile(tf.keras.optimizers.Adam(3e-6), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 26)]              0         
_________________________________________________________________
tf_bert_model_12 (TFBertMode ((None, 26, 768), (None,  109482240 
_________________________________________________________________
lambda_12 (Lambda)           (None, 768)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               98432     
_________________________________________________________________
dropout_493 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 129       
Total params: 109,580,801
Trainable params: 109,580,801
Non-trainable params: 0
_______________________________________

In [ ]:
history = model.fit(x = train_ids, y = train_labels, epochs = 1, verbose = 1, batch_size = 16, validation_data = (test_ids, test_labels))

495/495 [==============================] - 45s 91ms/step - loss: 0.1172 - accuracy: 0.9583 - val_loss: 0.0562 - val_accuracy: 1.0000


In [ ]:
import pandas as pd
ds = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AV/test_oJQbWVk.csv')

In [ ]:
sub_ids = ds.id.values
sub_tweets = ds.tweet.values
sub_tweets = preprocess(sub_tweets)
sub_tweets = encoder(sub_tweets)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predicted = model.predict(sub_tweets)

In [ ]:

predictions = [int(round(i[0])) for i in predicted]


In [ ]:
new_df = pd.DataFrame() 
new_df['id'] = sub_ids
new_df['label'] = predictions



In [ ]:
new_df.to_csv('./submission10d.csv',index = False)